# Data Cleaning
This notebook details the approach used to clean and generate features for the UK county health ranking dataset which can be found [here](https://public.tableau.com/app/learn/sample-data).

In [1]:
# Import library
import pandas as pd
import numpy as np
import plotly.express as px

First take a look at the data set:

In [2]:
df = pd.read_csv('data/County_Health_Rankings.csv')
display(df.head())
df.info()

,State,County,State code,County code,Year span,Measure name,Measure id,Numerator,Denominator,Raw value,Confidence Interval Lower Bound,Confidence Interval Upper Bound,Data Release Year,fipscode
0,US,United States,0.0,0.0,2003-2005,Violent crime rate,43.0,1328750.667,274877117.0,483.398066,NaN,NaN,NaN,0.0
1,US,United States,0.0,0.0,2004-2006,Violent crime rate,43.0,1340928.667,277612778.5,483.021233,NaN,NaN,NaN,0.0
2,US,United States,0.0,0.0,2005-2007,Violent crime rate,43.0,1355853.167,280407694.7,483.529230,NaN,NaN,2010.0,0.0
3,US,United States,0.0,0.0,2006-2008,Violent crime rate,43.0,1366928.333,287614567.7,475.263942,NaN,NaN,2011.0,0.0
4,US,United States,0.0,0.0,2007-2009,Violent crime rate,43.0,1339439.333,292576281.2,457.808585,NaN,NaN,2012.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303864 entries, 0 to 303863
Data columns (total 14 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   State                            303858 non-null  object 
 1   County                           303858 non-null  object 
 2   State code                       303860 non-null  float64
 3   County code                      303860 non-null  float64
 4   Year span                        303390 non-null  object 
 5   Measure name                     303390 non-null  object 
 6   Measure id                       303390 non-null  float64
 7   Numerator                        214076 non-null  float64
 8   Denominator                      184779 non-null  float64
 9   Raw value                        289956 non-null  float64
 10  Confidence Interval Lower Bound  189412 non-null  float64
 11  Confidence Interval Upper Bound  189412 non-null  float64
 12  Da

There are quite a large number of null values for the `Numerator`, `Denominator`, `Confidence Interval Lower Bound`, `Confidence Interval Upper Bound` and `Data Release Year` fields. First, we will explore the null values for the `State` field:

In [3]:
null_df = df[df['State'].isnull()]
null_df

,State,County,State code,County code,Year span,Measure name,Measure id,Numerator,Denominator,Raw value,Confidence Interval Lower Bound,Confidence Interval Upper Bound,Data Release Year,fipscode
157161,NaN,NaN,2.0,105.0,2008,Uninsured,85.0,600.0,1980.0,0.303,0.25596,0.35004,NaN,2105.0
157228,NaN,NaN,2.0,230.0,2008,Uninsured,85.0,153.0,792.0,0.193,0.15576,0.23024,NaN,2230.0
272420,NaN,NaN,NaN,NaN,1997-1999,Premature Death,1.0,97.0,NaN,9197.000,6605.90000,12467.30000,NaN,51560.0
272421,NaN,NaN,NaN,NaN,1998-2000,Premature Death,1.0,88.0,NaN,8781.000,6217.10000,12047.60000,NaN,51560.0
272422,NaN,NaN,NaN,NaN,1999-2001,Premature Death,1.0,52.0,NaN,NaN,NaN,NaN,NaN,51560.0
272423,NaN,NaN,NaN,NaN,2000-2002,Premature Death,1.0,23.0,NaN,NaN,NaN,NaN,NaN,51560.0


The rows with null values for `State` also has null values for `County`. The `State code`, `Country code` and `fipscode` fields should be useful in helping us determine the correct values for the missing `State` and `County`.

The FPIS Code was revised in 2006 and the former Skagway-Hoonah-Angoon Census Area was reconstituted as the Hoonah-Angoon Census Area when Skagway city consolidated into the Skagway Municipality county equivalent ([source](https://www.census.gov/programs-surveys/acs/technical-documentation/table-and-geography-changes/2009/geography-changes.html)). The county FIPS Codes 2105 and 2230 are merged into 2232.

Looking at the new Skagway-Hoonah-Angoon Census Area county in the dataset:

In [4]:
df[(df['State code'] == 2.0) & (df['County code'] == 232.0) & (df['Measure name'] =="Uninsured")]

,State,County,State code,County code,Year span,Measure name,Measure id,Numerator,Denominator,Raw value,Confidence Interval Lower Bound,Confidence Interval Upper Bound,Data Release Year,fipscode
157229,AK,Skagway-Hoonah-Angoon Census Area,2.0,232.0,2006,Uninsured,85.0,708.0,2751.0,0.258000,25.717680,0.340320,NaN,2232.0
157230,AK,Skagway-Hoonah-Angoon Census Area,2.0,232.0,2007,Uninsured,85.0,657.0,2712.0,0.242000,0.171440,0.312560,NaN,NaN
157231,AK,Skagway-Hoonah-Angoon Census Area,2.0,232.0,2009,Uninsured,85.0,805.0,2629.0,0.306200,NaN,NaN,2012.0,2232.0
157232,AK,Skagway-Hoonah-Angoon Census Area,2.0,232.0,2010,Uninsured,85.0,874.0,2785.0,0.313824,0.282261,0.345387,2013.0,2232.0
157233,AK,Skagway-Hoonah-Angoon Census Area,2.0,232.0,2011,Uninsured,85.0,811.0,2686.0,0.301936,0.260643,0.343229,2014.0,NaN


Indeed the value for 2008 is missing. We shall take the ratio of the sum of `Numerator` and `Denominator` to impute the missing `Raw value`:

In [5]:
df.loc[157161,'State'] = 'AK'
df.loc[157161,'County'] = 'Skagway-Hoonah-Angoon Census Area'
df.loc[157161,'Numerator'] = 600.0 + 153.0
df.loc[157161,'Denominator'] = 1980.0 + 792.0
df.loc[157161,'Raw value'] = (600.0 + 153.0)/1980.0 + 792.0
df.loc[157161,'State code'] = 2.0
df.loc[157161,'County code'] = 232.0
df.loc[157161,'fipscode'] = 2232.0

A quick search shows FIPS Code of 51560 ([source](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt)) is for Clifton Forge city in Alleghany County, Virginia. Looking through data to find if there is any missing values in Alleghany County, Virginia for premature death:

In [6]:
df[(df['County'] =="Alleghany County") & (df['Measure name'] =="Premature Death") & (df['State'] =="VA")]

,State,County,State code,County code,Year span,Measure name,Measure id,Numerator,Denominator,Raw value,Confidence Interval Lower Bound,Confidence Interval Upper Bound,Data Release Year,fipscode
271232,VA,Alleghany County,51.0,5.0,1997-1999,Premature Death,1.0,165.0,NaN,5750.500,4486.400,7260.300,NaN,51005.0
271233,VA,Alleghany County,51.0,5.0,1998-2000,Premature Death,1.0,168.0,NaN,5676.200,4441.400,7148.200,NaN,51005.0
271234,VA,Alleghany County,51.0,5.0,1999-2001,Premature Death,1.0,198.0,NaN,6231.000,5001.000,7672.000,NaN,51005.0
271235,VA,Alleghany County,51.0,5.0,2000-2002,Premature Death,1.0,224.0,NaN,6221.200,5038.300,7598.500,NaN,51005.0
271236,VA,Alleghany County,51.0,5.0,2001-2003,Premature Death,1.0,269.0,NaN,7773.300,6316.300,9465.700,NaN,51005.0
271237,VA,Alleghany County,51.0,5.0,2002-2004,Premature Death,1.0,279.0,NaN,8037.600,6468.100,9607.100,NaN,51005.0
271238,VA,Alleghany County,51.0,5.0,2003-2005,Premature Death,1.0,290.0,NaN,9218.300,7476.400,10960.200,NaN,51005.0
271239,VA,Alleghany County,51.0,5.0,2004-2006,Premature Death,1.0,292.0,NaN,8810.162,7207.376,10412.948,2010.0,51005.0
271240,VA,Alleghany County,51.0,5.0,2005-2007,Premature Death,1.0,313.0,NaN,10217.717,8384.592,12050.842,2011.0,51005.0
271241,VA,Alleghany County,51.0,5.0,2006-2008,Premature Death,1.0,325.0,NaN,10066.683,8289.433,11843.933,2012.0,51005.0


Seems like there is not missing data and since Clifton Forge city is not a state, we will remove these rows along with the errenous data in the previous part:

In [7]:
df = df.drop(df[df['State'].isnull()].index)

We also have rows with missing `Year span`:

In [8]:
df[df['Year span'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 99052 to 236823
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   State                            474 non-null    object 
 1   County                           474 non-null    object 
 2   State code                       474 non-null    float64
 3   County code                      474 non-null    float64
 4   Year span                        0 non-null      object 
 5   Measure name                     0 non-null      object 
 6   Measure id                       0 non-null      float64
 7   Numerator                        0 non-null      float64
 8   Denominator                      0 non-null      float64
 9   Raw value                        0 non-null      float64
 10  Confidence Interval Lower Bound  0 non-null      float64
 11  Confidence Interval Upper Bound  0 non-null      float64
 12  Data Release Ye

Since these data do not contain anything useful, we shall drop them:

In [9]:
null_year_and_raw_value_idx = df[(df['Year span'].isnull()) | (df['Raw value'].isnull())].index
df = df.drop(null_year_and_raw_value_idx)

Next we see if `County` name is unique for each set of `State` and `County code` value:

In [10]:
counts = df.groupby(['State', 'County code'], as_index = False)['County'].nunique()
counts[counts['County'] !=1]

,State,County code,County
756,IL,99.0,2


Seems like there is one county which do not have a unique county name for a given state of `State` and `County code`, let us find out what are these names:

In [11]:
il_99_df = df[(df['State'] == "IL") & (df['County code'] == 99)]
print(il_99_df['County'].unique())

['LaSalle County' 'La Salle County']


Looks like a spelling error which can be corrected easily:

In [12]:
df.loc[il_99_df.index, 'County'] = 'La Salle County'

The data also contained rows which are not conventional county i.e. those rows whose `State` is `US` and `County` is `United States`. Since these values cannot be mapped onto a US map and the calculation method for the `Raw value` differs from the rest of the data, we will remove those as well:

In [13]:
us_state_idx = df[df['State'] == 'US'].index
df.drop(us_state_idx, inplace = True)

Sort the dataframe and find the percentile of the `Raw value` for each county by `Measure name` and `Year span` and find the YoY change in percentile:

In [14]:
df = df.sort_values(['State', 'County', 'Measure name', 'Year span'])
df['Percentile'] = df.groupby(["Measure name", "Year span"])['Raw value'].rank(ascending = False, pct = True) * 100
df['Previous Time Period Percentile'] = df.groupby(["State", "County", "Measure name"])['Percentile'].shift()
df['Change in Percentile'] = round((df['Percentile'] - df['Previous Time Period Percentile'])/df['Previous Time Period Percentile'] * 100)
df['Change in Percentile'] = df['Change in Percentile'].fillna(0)

Changing the data types of some of the fields and taking a subset of features to be used in visualization:

In [15]:
df['State code'] = df['State code'].astype(int)
df['County code'] = df['County code'].astype(int)
df['Measure name'] = df['Measure name'].str.title()
df[['State', 'County', 'State code', 'County code', 'Year span', 'Measure name', 'Measure id', 'Raw value']]

,State,County,State code,County code,Year span,Measure name,Measure id,Raw value
214877,AK,Alaska,2,0,2004,Adult Obesity,11.0,0.250000
214878,AK,Alaska,2,0,2005,Adult Obesity,11.0,0.260000
214879,AK,Alaska,2,0,2006,Adult Obesity,11.0,0.270000
214880,AK,Alaska,2,0,2007,Adult Obesity,11.0,0.280000
214881,AK,Alaska,2,0,2008,Adult Obesity,11.0,0.280000
...,...,...,...,...,...,...,...,...
22185,WY,Wyoming,56,0,2005-2007,Violent Crime Rate,43.0,237.119564
22186,WY,Wyoming,56,0,2006-2008,Violent Crime Rate,43.0,237.568713
22187,WY,Wyoming,56,0,2007-2009,Violent Crime Rate,43.0,225.517941
22188,WY,Wyoming,56,0,2008-2010,Violent Crime Rate,43.0,208.744007


Saving the file:

In [16]:
df.to_csv('data/County_Health_Rankings_modified.csv', index = False)